In [2]:
%load_ext autoreload
%autoreload 2

In [37]:
from sklearn import set_config; set_config(display='diagram')
from package.etl import *
from package.train_baseline import TrainBaseline
from package.train_city import TrainCity
from package.train_store import TrainStore
from package.train_family import TrainFamily
import pandas as pd
import numpy as np
import requests
import joblib
import json

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

# Linear Baseline

In [4]:
X_train, y_train, X_predict, pipe, preprocessor = TrainFamily()
job = joblib.load('model_family.joblib')
job

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['year', 'month', 'city',
                                                   'store_nbr', 'family'])])),
                ('linearregression', LinearRegression())])

In [22]:
job.score(X_train,y_train)

0.6916893767585824

In [28]:
model = LinearRegression()
X_train_transformed = preprocessor.fit_transform(X_train)
cv_results = cross_validate(model, X_train_transformed, y_train, cv=5, 
                            scoring=['max_error',
                                     'r2', 
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error']
                           )
pd.DataFrame(cv_results)

,fit_time,score_time,test_max_error,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error
0,1.080794,0.004568,-1.029966e+01,5.416737e-01,-3.122166e+00,-1.205895e+01
1,0.957527,0.004221,-1.374009e+01,6.082059e-01,-2.192151e+00,-9.611896e+00
2,0.949363,0.003932,-1.345884e+01,6.080309e-01,-2.040599e+00,-8.511366e+00
3,0.958557,0.005850,-1.432212e+01,5.377893e-01,-1.856872e+00,-6.480984e+00
4,0.917941,0.011761,-2.621307e+13,-3.857945e+25,-1.872399e+13,-4.908133e+26


In [14]:
X = pd.DataFrame(dict(year=[2020],
             month=[1],
             city=['Quito'],
             store_nbr=[1],
             family=['AUTOMOTIVE']
            ))
X

,year,month,city,store_nbr,family
0,2020,1,Quito,1,AUTOMOTIVE


In [19]:
job.predict(X)[0]

2699654675541.9277

# Random Forest Regressor

In [39]:
model = RandomForestRegressor()

cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

preprocessor = ColumnTransformer([
                            ('cat_tr', cat_transformer, ['year', 'month', 'city', 'store_nbr', 'family'])
                        ],remainder='passthrough')

pipeline = make_pipeline(preprocessor, model)
pipeline.fit(X_train, y_train)
#joblib.dump(pipeline, 'model_rf_family.joblib')

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['year', 'month', 'city',
                                                   'store_nbr', 'family'])])),
                ('randomforestregressor', RandomForestRegressor())])

In [40]:
pipeline.score(X_train,y_train)

0.9945911117307488

In [41]:
X_train_transformed = preprocessor.fit_transform(X_train)
cv_results = cross_validate(model, X_train_transformed, y_train, cv=5, 
                            scoring=['max_error',
                                     'r2', 
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error']
                           )
pd.DataFrame(cv_results)

,fit_time,score_time,test_max_error,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error
0,89.576929,0.409244,-13.931602,0.885526,-0.686971,-3.011897
1,78.591043,0.413826,-14.060658,0.610611,-1.442380,-9.552883
2,68.787184,0.390159,-14.178850,0.527489,-1.425443,-10.260294
3,96.545820,0.486829,-13.573329,0.693323,-0.931220,-4.300142
4,101.488472,0.492813,-14.481667,0.245264,-1.549152,-9.601862


In [49]:
cv_results['test_neg_mean_squared_error'].mean()

-7.3454156422507335